<a href="https://colab.research.google.com/github/alwaysneedhelp/Practice-for-IOAI/blob/main/U_net_practice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Accepts 1-channel satellite images ([B, 1, H, W])

Predicts rain vs no-rain for each pixel ([B, 1, H, W])

Uses 3 encoder blocks (with MaxPool)

A bottleneck

3 decoder blocks (with ConvTranspose2d)

Skip connections via torch.cat


In [1]:
import torch
import torch.nn as nn

In [3]:
class UNetRainDetector(nn.Module):
    def __init__(self):
        super().__init__()

        self.encode1 = self.conv_block(1, 64)
        self.encode2 = self.conv_block(64, 128)
        self.encode3 = self.conv_block(128, 256)
        self.encode4 = self.conv_block(256, 512)

        self.mid = self.conv_block(512, 1024)

        self.pool = nn.MaxPool2d(2, 2)

        self.up1 = nn.ConvTranspose2d(1024, 512, 2, 2)
        self.dec1 = self.conv_block(1024, 512)

        self.up2 = nn.ConvTranspose2d(512, 256, 2, 2)
        self.dec2 = self.conv_block(512, 256)

        self.up3 = nn.ConvTranspose2d(256, 128, 2, 2)
        self.dec3 = self.conv_block(256, 128)

        self.up4 = nn.ConvTranspose2d(128, 64, 2, 2)
        self.dec4 = self.conv_block(128, 64)

        self.final = nn.Conv2d(64, 1, kernel_size=1)



    def conv_block(self, in_ch, out_ch):
        order = nn.Sequential([
            nn.Conv2d(in_ch, out_ch, 3, padding=1),
            nn.ReLU(inplace=True),
            nn.BatchNorm2d(out_ch),
            nn.Conv2d(out_ch, out_ch, 3, padding=1),
            nn.ReLU(inplace=True),
            nn.BatchNorm2d(out_ch),
        ])

        return order

    def forward(self, x):

        x1 = self.encode1(x)
        x2 = self.pool(self.encode2(x1))
        x3 = self.pool(self.encode3(x2))
        x4 = self.pool(self.encode4(x3))

        mid = self.mid(x4)

        d1 = self.dec1(torch.cat([self.up1(mid), x1], dim=1))
        d2 = self.dec2(torch.cat([self.up2(d1), x2], dim=1))
        d3 = self.dec3(torch.cat([self.up3(d2), x3], dim=1))
        d4 = self.dec4(torch.cat([self.up4(d3), x4], dim=1))

        return self.final(d4)